# Vanna 3.0 Quickstart
This notebook demonstrates how to set up a Vanna 3.0 agent with the new secure-by-default architecture, user-aware permissions, and the built-in FastAPI server.

In [ ]:
# Install Vanna with standard dependencies
!pip install 'vanna[fastapi,anthropic,duckdb]'

## 1. Setup Database
We'll download the sample Chinook database.

In [ ]:
import httpx

with open("Chinook.sqlite", "wb") as f:
    with httpx.stream("GET", "https://vanna.ai/Chinook.sqlite") as response:
        for chunk in response.iter_bytes():
            f.write(chunk)

## 2. Configure Vanna 3.0 Agent
We'll set up an agent with:
- **Anthropic Claude 3.5 Sonnet** as the LLM
- **SQLite** as the database runner
- **User Resolver** for auth simulation
- **Personalization** enabled (new in v3)

In [ ]:
import os
from vanna import Agent, AgentConfig
from vanna.servers.fastapi import VannaFastAPIServer
from vanna.core.registry import ToolRegistry
from vanna.core.user import UserResolver, User, RequestContext
from vanna.integrations.anthropic import AnthropicLlmService
from vanna.integrations.sqlite import SqliteRunner
from vanna.tools import RunSqlTool, VisualizeDataTool
from vanna.integrations.local.agent_memory import DemoAgentMemory

# Authorization logic (Simulation)
class SimpleUserResolver(UserResolver):
    async def resolve_user(self, request_context: RequestContext) -> User:
        # In a real app, you'd decode a JWT or check a session cookie here
        user_email = request_context.get_cookie('vanna_email') or "admin@example.com"
        
        print(f"Resolving user: {user_email}")
        groups = ['admin'] if "admin" in user_email else ['user']
        return User(id=user_email, email=user_email, group_memberships=groups, tenant_id="demo")

# Initialize Runner & Tools
runner = SqliteRunner(database_path="./Chinook.sqlite")
tools = ToolRegistry()

# Register standard tools with RBAC
tools.register_local_tool(RunSqlTool(sql_runner=runner), access_groups=['admin', 'user'])
tools.register_local_tool(VisualizeDataTool(), access_groups=['admin', 'user'])

# Set up LLM (Set ANTHROPIC_API_KEY env var or pass directly)
api_key = os.getenv("ANTHROPIC_API_KEY", "sk-ant-...") 
llm = AnthropicLlmService(model="claude-3-5-sonnet-20240620", api_key=api_key)

# Create the Agent
agent = Agent(
    llm_service=llm,
    tool_registry=tools,
    user_resolver=SimpleUserResolver(),
    agent_memory=DemoAgentMemory(max_items=1000),
    config=AgentConfig(
        enable_personalization=True,  # Enable v3 user profiles
        default_chart_style="vanna-dark"
    )
)

## 3. Run the Server
Start the built-in FastAPI server. This provides:
- Streaming chat endpoint `/api/vanna/v2/chat_sse` (v3 compatible backend)
- Static web UI at `/`

In [ ]:
server = VannaFastAPIServer(agent)
print("Starting Vanna 3.0 server on http://localhost:8000 ...")
server.run(port=8000)